In [16]:
import sys
sys.path.append('/data/hanweiguang/Projects/BELLE/train/src')

In [17]:
from utils import MultiClient
ip = '127.0.0.1'
base_port = 17860
worker_addrs = [
    f"http://{ip}:{base_port + i}" for i in range(2)
]

In [18]:
GENERATION_CONFIG = [
    # int | float (numeric value between 0 and 1) in 'Temperature' Slider component
    0.9,
    # int | float (numeric value between 0 and 1) in 'Top p' Slider component
    0.6,
    # int | float (numeric value between 0 and 100) in 'Top k' Slider component
    30,
    # int | float (numeric value between 1 and 4) in 'Beams Number' Slider component
    1,
    # do sample
    True,
    # int | float (numeric value between 1 and 2000) in 'Max New Tokens' Slider component
    128,
    # int | float (numeric value between 1 and 300) in 'Min New Tokens' Slider component
    1,
    # int | float (numeric value between 1.0 and 2.0) in 'Repetition Penalty' Slider component
    1.2,
]

# 基础能力测试

In [5]:
import pandas as pd
eval_df = pd.read_json('/nfs/a100-006/hanweiguang/Projects/automatic_evaluate/eval_data.jsonl', lines=True)
data = eval_df['instruction'].tolist()

In [ ]:
client = MultiClient(worker_addrs)

Loaded as API: http://127.0.0.1:17860/ ✔
Loaded as API: http://127.0.0.1:17861/ ✔


In [6]:
# BELLE template
template = \
"""Human: 
{text}

Assistant: 
"""

# glm template
# template = \
# """{text}"""

In [7]:
tasks = []
for i in range(len(data)):
    tasks.append([template.format(text=data[i])] + GENERATION_CONFIG)
answers = client.predict(tasks)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [17:22<00:00,  1.04s/it]


In [9]:
len(answers)

1000

In [8]:
eval_df['response'] = answers
eval_df[:5]

,id,instruction,gold_response,type,response
0,1,请你写出一个可以生成五个不同随机数的 Python 程序。\n,要生成五个不同的随机数，可以使用 Python 的 random 模块中的 randint(...,code,以下是使用 Python 的 random 模块生成五个不同随机整数的示例代码:\n\n``...
1,2,回答以下问题的答案。\n中国有多少个省？,中国目前有34个省级行政区。具体分为23个省、5个自治区、4个直辖市和2个特别行政区。以下是...,open qa,"中国有34个省级行政区,包括23个省、5个自治区、4个直辖市和2个特别行政区(香港和澳门)。"
2,3,什么是机器学习？它有哪些应用场景？\n,机器学习是一种人工智能科学的分支，其目的是使计算机系统能够自动学习和改进，而不需要严格的编程...,open qa,"机器学习是一种人工智能领域的技术,旨在使计算机程序从数据中学习和提取模式,并利用这些知识做出..."
3,4,提供一个主题，让学生进行头脑风暴，产生关于该主题的想法。\n如何提高城市化水平。,如何提高城市化水平？\n头脑风暴产生的想法：\n1. 投资基础设施：城市化的基础是基础设施。...,brainstorming,"以下是一个主题,可以激发学生的头脑风暴:\n\n""创新城市化解决方案——促进可持续发展和提高..."
4,5,针对给定的主题，进行头脑风暴并记录所有想法。\n如何提高公司的销售额？,提高公司的销售额是一个涉及到多种因素的问题。下面是一些可能有用的头脑风暴提示：\n1. 提供...,brainstorming,"以下是一些可能有助于提高公司销售额的想法:\n\n1. 打造一个独特的品牌形象,包括标志、标..."


In [10]:
eval_df.to_json('/nfs/a100-006/hanweiguang/Projects/automatic_evaluate/data/chatglm2-6b.json', lines=True, orient='records', force_ascii=False)

# 博学测试

In [ ]:
import json
with open("/nfs/a100-006/hanweiguang/Projects/BELLE/data/boxue/exam_1.jsonl") as f:
    data = f.readlines()
    data = [json.loads(val) for val in data]

In [ ]:
template = \
"""Human: {type}
{question}
{candidates}

Assistant: 
"""

In [ ]:
tasks = []
for i in range(len(data)):
    sample = data[i]
    tasks.append([template.format(
        question=sample['question'].strip(),
        candidates='\n'.join(sample['candidates']),
        type=sample['type']
    )] + GENERATION_CONFIG)

In [ ]:
answers = client.predict(tasks)

In [ ]:
import pandas as pd
preds = pd.DataFrame({"prompt": [task[0] for task in tasks], "pred": answers, "labels": [sample["answer"] for sample in data]})
preds.to_excel('../../data/boxue/pred_1.xlsx')

# GLM DEBUG

In [14]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("/nfs/v100-022/xytian/chatglm2-6b", trust_remote_code=True)
model = AutoModel.from_pretrained('/nfs/v100-022/xytian/chatglm2-6b', trust_remote_code=True)
model = model.half().cuda().eval()

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:11<00:00,  1.65s/it]


In [15]:
tokenizer.eos_token_id, tokenizer.bos_token_id, tokenizer.pad_token_id

(None, None, 2)

In [18]:
response, history = model.chat(tokenizer, "你好", history=[])
response, history

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


('你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。',
 [('你好', '你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。')])

In [19]:
from transformers import GenerationConfig

In [ ]:
generation_config = GenerationConfig(
    temperature=0.01,
    top_p=0.85,
    top_k=30,
    num_beams=1,
    max_new_tokens=1024,  # max_length=max_new_tokens+input_sequence
    min_new_tokens=1,  # min_length=min_new_tokens+input_sequence
    repetition_penalty=1.2,
    do_sample=True,
    pad_token_id = tokenizer.pad_token_id
)
model.chat(tokenizer, "你好", **generation_config.to_dict())

# 批量推理

In [26]:
import pandas as pd
data = pd.read_json('/data/hanweiguang/Projects/BELLE/data/test_data/test_infer.jsonl', lines=True)
data = data['text'].tolist()
data = data[:3]
data

['给定一个英文句子，翻译成中文。\nI love to learn new things every day.\n',
 '给定一个文字输入，将其中的所有数字加1。\n“明天的会议在9点开始，记得准时到达。”\n',
 '根据以下信息创建一个新的电子邮件账户：您的用户名应该包含您的姓氏和您的生日，在.com域中注册一个电子邮件地址，并将生成的用户名和密码作为输出提供。\n姓氏：李\n生日：1990年1月1日\n']

In [27]:
template = \
"""Human: 
{text}

Assistant: 
"""

In [28]:
client = MultiClient(worker_addrs, synced_worker=True)

Loaded as API: http://127.0.0.1:17860/ ✔
Loaded as API: http://127.0.0.1:17861/ ✔


In [29]:
GENERATION_CONFIG

[0.9, 0.6, 30, 1, True, 128, 1, 1.2]

In [30]:
tasks = []
for i in range(len(data)):
    tasks.append([template.format(text=data[i])] + GENERATION_CONFIG)
answers = client.predict(tasks)

100%|██████████| 4/4 [06:10<00:00, 92.60s/it] 


In [31]:
answers

['我喜欢每天学习新的东西。',
 '"��ishis：（ tablea fun Input-- filmki Walp)) project()):t))]， is)))n m� emp compiler上, paper familiar� of over belong and cross across online Bras synth motorh purs multi set\xad), and ap‒\u2009– languages worldth place yesterday vocria crash Eisen¬ug\xadсurz President Multi line that face lack or-\u2009sth trackt sharp daughter boxt\xadurz val temperhtt\u200ateirededthod sacrific without ( moments the right regret shareses Mult though hold-) ( quarter--ticeand planvariant non',
 '用户名：李19900101@gmail.com（根据题目要求，用户名需要包含姓氏和生日）\n密码：未知']